In [2]:
import requests
import json
import pandas as pd
from pandas import json_normalize 
import numpy as np




In [3]:
# read and clean data


def correct_nested_strings(x):
    import re
    import numpy as np
    x = re.sub(r'\W*(na)*\W*(na)+\W*',' ',x).strip()
    if x != '':
        if (len(x.split(" ")) ==1) & (re.match(r',',x)==False):
            return x
        else:
            x = re.sub(r',', " ", x)
            d = x.split(" ")
            d = np.array(d)
        
            d = np.unique(d).tolist()
            return ','.join(d)
    else: 
        return None
# read data
recipe_data=pd.read_csv('recipe_links.csv')  

# clean
recipe_data= recipe_data.groupby(['title', 'recipe_link', 'img_link', 'ingredients', 'yield', 'nutrients']).agg({'meal_type': lambda x: ",".join(x), 'subregion':  lambda x: ",".join(x), 'region':  lambda x: ",".join(x)}).reset_index()
recipe_data[['region', 'subregion', 'meal_type']] = recipe_data[['region', 'subregion', 'meal_type']].applymap(correct_nested_strings)
recipe_data

,title,recipe_link,img_link,ingredients,yield,nutrients,meal_type,subregion,region
0,Sweetest Southern Sweet Potato Pie,https://www.allrecipes.com/recipe/228449/sweet...,https://www.allrecipes.com/thmb/1FQpSI5ZQHr2h6...,"['2 sweet potatoes', '2 cups white sugar', '0....",8 servings,"{'calories': '530 kcal', 'carbohydrateContent'...",dessert,southern,usa
1,Swiss Almond Carrot Cake (Aargauer Rueblitorte),https://www.allrecipes.com/recipe/262947/swiss...,https://www.allrecipes.com/thmb/KeeQElqXMoGWmb...,"['1 teaspoon butter', '4 eggs, separated', '1 ...",10 servings,"{'calories': '439 kcal', 'carbohydrateContent'...",None,swiss,european
2,Swiss Fondue,https://www.allrecipes.com/recipe/285711/swiss...,https://www.allrecipes.com/thmb/9ZlJwsHO2r1s0s...,"['2.5 cups shredded Gruyere cheese', '2.5 cups...",4 servings,"{'calories': '902 kcal', 'carbohydrateContent'...",None,swiss,european
3,Swiss Steak Quick and Easy,https://www.allrecipes.com/recipe/85934/swiss-...,https://www.allrecipes.com/thmb/WbV_HJYS-1SD0R...,"['1 tablespoon vegetable oil', '2 pounds cube ...",6 servings,"{'calories': '471 kcal', 'carbohydrateContent'...",None,swiss,european
4,Sylvia's Easy Greek Salad,https://www.allrecipes.com/recipe/14315/sylvia...,https://www.allrecipes.com/thmb/s8I8M6j31i0O6W...,"['0.75 cup red bell pepper, chopped', '0.75 cu...",4 servings,"{'calories': '331 kcal', 'carbohydrateContent'...",salad,greek,european
...,...,...,...,...,...,...,...,...,...
685,Zucchini with Chickpea and Mushroom Stuffing,https://www.allrecipes.com/recipe/25920/zucchi...,https://www.allrecipes.com/thmb/savfWtmsiRFVwi...,"['4 zucchini, halved', '1 tablespoon olive oil...",8 servings,"{'calories': '107 kcal', 'carbohydrateContent'...",main,jewish,usa
686,Zucchini with Dill Weed and Garlic-Yogurt Sauce,https://www.allrecipes.com/recipe/200196/zucch...,https://www.allrecipes.com/thmb/4S2mv4ef9kDAiD...,"['0.25 cup olive oil', '1 onion, finely choppe...",4 servings,"{'calories': '292 kcal', 'carbohydrateContent'...",None,turkish,middle_eastern
687,Zuppa Di Pesce Fra Di Avolo,https://www.allrecipes.com/recipe/230551/zuppa...,https://www.allrecipes.com/thmb/xcwpHccFQPzZ2w...,"['3 tablespoons olive oil', '2 pounds sea scal...",10 servings,"{'calories': '775 kcal', 'carbohydrateContent'...",None,italian,european
688,Zuppa Toscana,https://www.allrecipes.com/recipe/13126/zuppa-...,https://www.allrecipes.com/thmb/MKX7wl28t6i3Do...,"['1 (16 ounce) package smoked sausage', '2 pot...",6 servings,"{'calories': '459 kcal', 'carbohydrateContent'...",soups_and_stews,italian,european


In [13]:
# make request
def get_ingredient(x):
    return x.replace('[', "").replace(']',"").replace('(', '').replace(')',"").replace("'", '').replace("'", '')
#     return ', '.join([i.replace("[","").replace("]","") for i in x])

def get_nutrition(df):
    # set up request to get nutrition per 
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'

    headers = {
                'Content-Type': 'application/x-www-form-urlencoded',
                'x-app-id': api_id3,
                'x-app-key': api_key3,
                'x-remote-user-id': '0'
    }
    
    data = df['ingredients'].apply(get_ingredient).tolist()
    
    #get nutrition by recipe
    nutrition_list = []
    for i in data:
        
        # make request
        query = {
            "query": i
        }
        response = requests.request("POST", url, headers=headers, data=query)
        temp_data = json_normalize(response.json(), 'foods')
        
        # get certain nutri
        calories = temp_data['nf_calories'].sum()
        protein = temp_data['nf_protein'].sum()
        fat = temp_data['nf_total_fat'].sum()
        carb = temp_data['nf_total_carbohydrate'].sum()
        nutrition_list.append([calories,protein,fat,carb])
        
    nutrition_df = pd.DataFrame(nutrition_list, columns=['total_calories', 'total_protein', 'total_fat', 'total_carb'])
    return nutrition_df

# can only get 200 each time so I have to rename it to keep track
cal_data_600_799= get_nutrition(recipe_data.loc[600:799])
cal_data_600_799

,total_calories,total_protein,total_fat,total_carb
0,1422.79,38.57,70.07,171.81
1,3485.34,165.48,157.74,341.28
2,455.80,31.76,25.62,25.36
3,776.49,17.63,49.01,75.55
4,3811.97,197.80,262.01,162.62
...,...,...,...,...
85,1002.07,47.52,31.26,150.35
86,1159.57,39.18,64.67,114.46
87,12607.57,1580.81,311.36,781.72
88,2556.98,99.42,183.58,132.14


In [14]:
df_lst = [cal_data_0_199,cal_data_200_399,cal_data_400_599,cal_data_600_799]
new_df = pd.concat(df_lst).reset_index(drop=True)
new_df

,total_calories,total_protein,total_fat,total_carb
0,4371.41,45.81,181.28,657.11
1,4353.70,105.81,204.15,573.71
2,3506.82,174.08,196.23,199.73
3,3218.34,273.37,156.92,167.90
4,1327.64,15.96,127.81,37.74
...,...,...,...,...
685,1002.07,47.52,31.26,150.35
686,1159.57,39.18,64.67,114.46
687,12607.57,1580.81,311.36,781.72
688,2556.98,99.42,183.58,132.14


In [15]:
# merge back to the dataset
d = recipe_data.join(new_df)
d.to_csv('cal_data_6.csv', index = False)